In [ ]:
import pandas as pd
import geopandas as gpd
import folium

# read the shape file
#sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA94.shp")
sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA94.shp")
sf = sf.iloc[672:1366]

In [ ]:
# Convert the geometry shaape to to latitude and longitude
# Please attribute this if you are using it
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf.rename(columns={'POA_CODE21': 'Postcode'})

In [ ]:
# create a JSON 
geoJSON = sf[['Postcode', 'geometry']].drop_duplicates('Postcode').to_json()


In [ ]:
#_map = folium.Map(location=[143.73, -37.56], tiles="Stamen Terrain", zoom_start=10)
# refer to the folium documentations on how to plot aggregated data.
#_map.add_child(folium.Choropleth(
    #geo_data=geoJSON,
    #name='choropleth',
#))

In [ ]:
# income Distribution

income = pd.read_csv('../data/curated/predict_income.csv')

income = income[['Postcode', '2022']]

postcode = sf['Postcode'].astype(int).astype('str')
income['Postcode'] = income['Postcode'].astype(int).astype('str')

income = pd.merge(income, postcode, on='Postcode', how='right')

In [ ]:
# plot the counts on geo map
m = folium.Map(location=[-36.9848, 143.3906], tiles="Stamen Terrain", zoom_start=10)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=income, # data source
    columns=['Postcode','2022'], # the columns required
    key_on='properties.Postcode', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='white',
    legend_name='Income Distribution'
)

c.add_to(m)

# save the map
m.save('../plots/income distribution.html')

m